# Getting known with keras

That is the first attempt in using the keras library, so here I am just getting familiar with what it has

## Emotion recognition

The dataset has labelled 64x64 face images, the labels are 0="not smiling" and 1="smiling". The task is given the face to detect whether the person is smiling or not

In [8]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import h5py

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

## Loading data

In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_happy.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_happy.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [5]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


## Creating the model

In [6]:
def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset
        (height, width, channels) as a tuple
    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(32, (7, 7), strides=(1, 1), name='conv0')(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(inputs=X_input, outputs=X, name='HappyModel')
    
    return model

## Creating and training the model

In [7]:
happyModel = HappyModel(X_train[0].shape)

In [17]:
opt = keras.optimizers.Adam(learning_rate=0.01)
happyModel.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [18]:
happyModel.fit(X_train, Y_train, batch_size=64, epochs=100)

Epoch 1/100
10/10 [==============================] - 6s 574ms/step - loss: 3.5302 - accuracy: 0.7800
Epoch 2/100
10/10 [==============================] - 4s 390ms/step - loss: 0.4157 - accuracy: 0.9183
Epoch 3/100
10/10 [==============================] - 4s 389ms/step - loss: 0.2430 - accuracy: 0.95000s - loss: 0.2430 - accuracy: 0.95
Epoch 4/100
10/10 [==============================] - 4s 388ms/step - loss: 0.0548 - accuracy: 0.9850
Epoch 5/100
10/10 [==============================] - 4s 434ms/step - loss: 0.0253 - accuracy: 0.9883
Epoch 6/100
10/10 [==============================] - 4s 412ms/step - loss: 0.0256 - accuracy: 0.9883
Epoch 7/100
10/10 [==============================] - 4s 395ms/step - loss: 0.0208 - accuracy: 0.9933
Epoch 8/100
10/10 [==============================] - 4s 389ms/step - loss: 0.0217 - accuracy: 0.9917
Epoch 9/100
10/10 [==============================] - 4s 388ms/step - loss: 0.0472 - accuracy: 0.9833
Epoch 10/100
10/10 [==============================] - 4s 

10/10 [==============================] - 4s 394ms/step - loss: 4.0666e-04 - accuracy: 1.0000
Epoch 81/100
10/10 [==============================] - 4s 397ms/step - loss: 4.0324e-04 - accuracy: 1.0000
Epoch 82/100
10/10 [==============================] - 4s 403ms/step - loss: 3.7799e-04 - accuracy: 1.0000
Epoch 83/100
10/10 [==============================] - 4s 389ms/step - loss: 3.5085e-04 - accuracy: 1.0000
Epoch 84/100
10/10 [==============================] - 4s 388ms/step - loss: 4.1833e-04 - accuracy: 1.0000
Epoch 85/100
10/10 [==============================] - 4s 402ms/step - loss: 3.5816e-04 - accuracy: 1.0000
Epoch 86/100
10/10 [==============================] - 4s 392ms/step - loss: 3.7022e-04 - accuracy: 1.0000
Epoch 87/100
10/10 [==============================] - 4s 413ms/step - loss: 3.6076e-04 - accuracy: 1.0000
Epoch 88/100
10/10 [==============================] - 4s 390ms/step - loss: 3.8414e-04 - accuracy: 1.0000
Epoch 89/100
10/10 [==============================] - 4s 39

## Calculating the accuracy

In [19]:
preds = happyModel.evaluate(X_test, Y_test, return_dict=True)

print(preds)

5/5 [==============================] - 0s 96ms/step - loss: 0.0243 - accuracy: 0.9867
{'loss': 0.024342834949493408, 'accuracy': 0.9866666793823242}
